### Importing Libraries

In [20]:
import io
import random
import string
import warnings
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')


import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *


# sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics


# python imports
import re
import json
import os
from collections import Counter
import datetime as dt
from dateutil.parser import parse


# Visualization
from matplotlib import pyplot as plt
from matplotlib import ticker
import seaborn as sns
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from wordcloud import WordCloud
from tqdm import tqdm_notebook
import geopandas


# twitter
# pip install twarc
from twarc import Twarc
import carmen
import pycountry as pc

resolver = carmen.get_resolver()
resolver.load_locations()


# Saving models
import pickle

import config

### Reading the Data

In [ ]:
main_dir = '/home/vca_rishik/rishik/COVID-19-tweets/'
full_dir = main_dir + 'data_full/'
id_dir   = main_dir + 'data/'

popularity_list = []
text_list = []
entities_list = []

for file in sorted(os.listdir(full_dir))[:-2]:
    print('reading... ' + file)
    
    df_full = pd.read_csv(full_dir + file).drop_duplicates()

    text_list.append(df_full[['id', 'full_text']])
    popularity_list.append(df_full[['id', 'retweet_count', 'favorite_count']])
    entities_list.append(df_full[['id', 'entities']])
    
    print(file + ' ..done')
    print()

In [114]:
a = (eval(entities_list[1].iloc[1].entities))
a.get('hashtags')[0].get('text')

'COVID19'

In [132]:
def getHashtags(entity):
    if(isinstance(entity, str)):
        parsed = eval(entity)
        list_tags = [tag.get('text') for tag in parsed.get('hashtags')]
        return list_tags
    
    print(entity)
    return []

In [115]:
df_entities = pd.concat(entities_list)
df_entities.head()

,id,entities
0,1.256010e+18,"{'hashtags': [{'text': 'SequoiaHR', 'indices':..."
1,1.256010e+18,"{'hashtags': [{'text': 'COVID19', 'indices': [..."
2,1.256010e+18,"{'hashtags': [{'text': 'keepseniorssafe', 'ind..."
3,1.256010e+18,"{'hashtags': [{'text': 'coronavirus', 'indices..."
4,1.256010e+18,"{'hashtags': [{'text': 'China', 'indices': [17..."


In [133]:
df_entities['hashtags'] = df_entities.entities.apply(getHashtags)
df_entities

nan
nan
nan
nan
nan
nan
nan
nan
nan


,id,entities,hashtags
0,1.256010e+18,"{'hashtags': [{'text': 'SequoiaHR', 'indices':...","[SequoiaHR, HR, Business, COVID19]"
1,1.256010e+18,"{'hashtags': [{'text': 'COVID19', 'indices': [...","[COVID19, ncdc]"
2,1.256010e+18,"{'hashtags': [{'text': 'keepseniorssafe', 'ind...","[keepseniorssafe, Bracebridge, COVID19, corona..."
3,1.256010e+18,"{'hashtags': [{'text': 'coronavirus', 'indices...",[coronavirus]
4,1.256010e+18,"{'hashtags': [{'text': 'China', 'indices': [17...","[China, COVID19]"
5,1.256010e+18,"{'hashtags': [{'text': 'coronavirus', 'indices...",[coronavirus]
6,1.256010e+18,"{'hashtags': [{'text': 'QuarantineLifeBeLike',...","[QuarantineLifeBeLike, Job, QuarantineLife, lo..."
7,1.256010e+18,"{'hashtags': [{'text': 'Huduma', 'indices': [4...","[Huduma, Kenya, COVID19]"
8,1.256010e+18,"{'hashtags': [{'text': 'privateequity', 'indic...","[privateequity, coronavirus, newsletter]"
9,1.256010e+18,"{'hashtags': [{'text': 'COVID19', 'indices': [...","[COVID19, Pakistan, terrorism]"
